In [6]:
from kerchunk.hdf import SingleHdf5ToZarr 
from kerchunk.combine import MultiZarrToZarr
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import s3fs
import datetime as dt
import logging
import fsspec
import ujson
from tqdm import tqdm
from glob import glob

In [7]:
fs = fsspec.filesystem('s3', anon=True)

In [8]:
urls = ['s3://' + f for f in fs.glob("s3://noaa-goes16/ABI-L2-SSTF/2020/210/*/*.nc")]

In [9]:
import dask

In [10]:
from dask.distributed import Client

In [11]:
client = Client(n_workers=30)

In [12]:
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 30
Total threads: 30,Total memory: 15.32 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:33077,Workers: 30
Dashboard: http://127.0.0.1:8787/status,Total threads: 30
Started: Just now,Total memory: 15.32 GiB
Comm: tcp://127.0.0.1:39851,Total threads: 1
Dashboard: http://127.0.0.1:40553/status,Memory: 522.93 MiB
Nanny: tcp://127.0.0.1:42593,


In [19]:
def gen_json(u):
    so = dict(
        mode="rb", anon=True, default_fill_cache=False,
        default_cache_type="none"
    )
    with fsspec.open(u, **so) as inf:
        h5chunks = SingleHdf5ToZarr(inf, u, inline_threshold=300)
        with open(f"tests/{u.split('/')[-1]}.json", 'wb') as outf:
           outf.write(ujson.dumps(h5chunks.translate()).encode())

In [15]:
import pathlib

In [16]:
pathlib.Path('./tests/').mkdir(exist_ok=True)

In [20]:
dask.compute(*[dask.delayed(gen_json)(u) for u in urls])

2024-03-04 03:13:17,663 - distributed.scheduler - WARNING - Worker failed to heartbeat within 300 seconds. Closing: <WorkerState 'tcp://127.0.0.1:38337', name: 3, status: running, memory: 0, processing: 1>
2024-03-04 03:13:35,869 - distributed.nanny - WARNING - Worker process still alive after 3.1999989318847657 seconds, killing


(None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None)